## title

**paper:** []() - 

**date, curator:** 2024-0, Sara Carsanaro

### annotation summary
run this after annotation is complete

In [ ]:
anat_summary = library_to_add[['infoOrgan', 'anatId', 'anatName', 'anatAnnotationStatus']]
unique_anat = anat_summary.drop_duplicates()
display_df(unique_anat)

In [ ]:
dev_summary = library_to_add[['infoStage', 'stageId', 'stageName', 'stageAnnotationStatus']]
unique_dev = dev_summary.drop_duplicates()
display_df(unique_dev)

### set variables, import packages, define functions

In [19]:
experiment_id = ""

path_to_create_exp_script = "/Users/scarsana/Desktop/git/scRNA-Seq/scripts/Create_ExpLib_tables.py" 
experiment_type = "bulk"

path_to_output_main = "/Users/scarsana/Desktop/git/expression-annotations/Notebooks/bulk/" 
path_to_output = "{}{}/".format(path_to_output_main, experiment_id)
library_path_from_script = "{}RNASeqLibrary_{}.tsv".format(path_to_output, experiment_id)
experiment_path_from_script = "{}RNASeqExperiment_{}.tsv".format(path_to_output, experiment_id)
library_to_add_path = "{}complete_RNASeqLibrary_{}.tsv".format(path_to_output, experiment_id)
experiment_to_add_path = "{}complete_RNASeqExperiment_{}.tsv".format(path_to_output, experiment_id)
script_file = "{}.ipynb".format(experiment_id)
commit_message_exp = '"adding annotated bulk experiment {}"'.format(experiment_id)
commit_message_py = '"adding annotation files for {} to notebook folder"'.format(experiment_id)


## to add to git
path_to_git_annotations = "/Users/scarsana/Desktop/git/expression-annotations/RNA_Seq/"
git_library_path = "{}RNASeqLibrary.tsv".format(path_to_git_annotations)
git_experiment_path = "{}RNASeqExperiment.tsv".format(path_to_git_annotations)

library_cols = ['#libraryId', 'experimentId', 'platform', 'SRSId', 'anatId', 'anatName', 'stageId', 'stageName', 'url_GSM', 'infoOrgan', 'infoStage', 'anatAnnotationStatus', 'anatBiologicalStatus', 'stageAnnotationStatus', 'sex', 'strain', 'genotype', 'speciesId', 'protocol', 'protocolType', 'RNASelection', 'globin_reduction', 'replicate', 'lib_name', 'sampleName', 'sampleAge_value', 'sampleAge_unit', 'PATOid', 'PATOname','comment', 'condition', 'physiologicalStatus', 'annotatorId', 'lastModificationDate']

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import os
import csv

# displays df with the scrollbar next to the DataFrame
def display_df(df):
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    display(HTML("<div style='height: 300px; overflow: auto; width: fit-content'>" +
        df.style.to_html(index=False) + "</div>"))

# function that compares two columns in a dataframe and tells you which ones are not equal (case insensitive)
def compare_columns(df, col1, col2, return_col):
    compare_return = df[col1].str.lower() != df[col2].str.lower()  
    df.loc[compare_return, return_col] 
    if not any(compare_return):
        print("The two columns are equal (case insensitive)")
    else:
        print("The following rows are not equal: ")
        print(df.loc[compare_return, return_col])

# fixes formatting of file to match libreoffice settings/historic file format
def update_format(path):
    with open(path, 'r') as file:
        filedata = file.read()
    # Replace the target string
    filedata = filedata.replace("\t\"\"", "\t")
    # Write the file out again
    with open(path, 'w') as file:
        file.write(filedata)

# checks for duplicate values in a specific column and prints those values + the corresponding library id
def dup_check(df, column):
    duplicateCheck = df.duplicated(subset=[column], keep=False)
    duplicateCheck.sort_values(inplace=True)
    if duplicateCheck.unique().any() == False:
        print("no duplicate values in " + column)
    elif duplicateCheck.unique().any() == True and column != '#libraryId':
        dups = df[duplicateCheck].loc[:,['#libraryId', column]]
        df_dups = pd.DataFrame(dups)
        df_dups.sort_values(inplace=True, by=column)
        print(df_dups)
    elif duplicateCheck.unique().any() == True and column == '#libraryId':
        print(df[duplicateCheck].loc[:,['#libraryId']])

# prints all unique values in a specific column
def unique_sorted(df, column):
    unique = df[column].unique()
    unique.sort()
    print(unique)

### script

In [ ]:
! python3 $path_to_create_exp_script $experiment_id $path_to_output $experiment_type

### library annnotations

In [ ]:
library = pd.read_csv(library_path_from_script, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN'], dtype=object)
display_df(library)

#### anatomical entity
* [uberon ols](https://www.ebi.ac.uk/ols4/ontologies/uberon)

In [ ]:
unique_sorted(library, "infoOrgan")

In [ ]:

# all
library.loc[:,'anatId'] = ''
library.loc[:,'anatName'] = ''
# perfect match, missing child term, other
library.loc[:,'anatAnnotationStatus'] = ''
# partial sampling, full sampling, not documented
library.loc[:,'anatBiologicalStatus'] = ''

# conditional (based off infoOrgan)
library.loc[library["infoOrgan"] == "old", "anatId"] = "new"
library.loc[library["infoOrgan"] == "old", "anatName"] = "new"
# perfect match, missing child term, other
library.loc[library["infoOrgan"] == "old", "anatAnnotationStatus"] = "new"
library.loc[library["infoOrgan"] == "old", "anatBiologicalStatus"] = "new"

# view
display_df(library)

#### stage
- [species specific developmental ontologies](https://github.com/obophenotype/developmental-stage-ontologies/tree/master/src)

In [ ]:
unique_sorted(library, "infoStage")

In [ ]:
# all
library.loc[:,'stageId'] = ''
library.loc[:,'stageName'] = ''
# perfect match, missing child term, other
library.loc[:,'stageAnnotationStatus'] = ''

# conditional (based off infoStage)
library.loc[library["infoStage"] == "old", "stageId"] = "new"
library.loc[library["infoStage"] == "old", "stageName"] = "new"
# perfect match, missing child term, other
library.loc[library["infoStage"] == "old", "stageAnnotationStatus"] = "new"

# view
display_df(library)

#### sex, strain, genotype, speciesId
- uniprot [strain list](https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/docs/strains)
- uniprot [species list](https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/docs/speclist)
- bgee [strain mapping](https://gitlab.sib.swiss/Bgee/expression-annotations/-/tree/develop/Strains?ref_type=heads)

In [ ]:
library.loc[library["sex"] == "male", "sex"] = "M"
library.loc[library["sex"] == "female", "sex"] = "F"

#library.loc[:,'strain'] = ''

#library.loc[:,'genotype'] = ''

#library.loc[:,'speciesId'] = ''

# view
display_df(library)

#### protocol
see [bulk kits](https://gitlab.sib.swiss/Bgee/scRNA-Seq/-/blob/main/scripts/bulk_kits.csv) for some common protocols

In [ ]:
# making these variables because we use them again in the experiment file
my_protocol = ''
# full_length or 3'
my_protocolType = ''

library.loc[:,'protocol'] = my_protocol
library.loc[:,'protocolType'] = my_protocolType
# polyA, ribo-minus, miRNA, lncRNA, circRNA
library.loc[:,'RNASelection'] = ''

# view
display_df(library)

#### globin, replicates

In [ ]:
# check for duplicate SRSId values
dup_check(library, "SRSId")

In [ ]:
#library.loc[:,'globin_reduction'] = 'Y'

# replicates
#library.loc[library["#libraryId"] == "old", "replicate"] = "1"
#library.loc[library["#libraryId"].isin(["one", "two"]), "replicate"] = "1"

# view
display_df(library)

#### sample age, pato, physiological status

In [ ]:
#library.loc[:,'sampleAge_value'] = ''
#library.loc[:,'sampleAge_unit'] = ''

# ex. castrated male
#library.loc[:,'PATOid'] = ''
#library.loc[:,'PATOname'] = ''

# ex. castrated, pregnant, pre-smoltification, post-smoltification, laying eggs
#library.loc[:,'physiologicalStatus'] = ''

# view
display_df(library)

#### condition

In [ ]:
# ex. control, diet, light, reproductive capacity, time post mortem, time post feeding, 
# exercise details, menstruation, personality, litter size 
#library.loc[library["condition"] == "old", "condition"] = "new"

# view
display_df(library)

#### annotator id, last modification date

In [ ]:
library.loc[:,'annotatorId'] = 'SAC'
library.loc[:,'lastModificationDate'] = '2024-0'

# view
display_df(library)

#### comments

In [ ]:
#library.loc[:,'comment'] = ''

#### save complete file with correct columns

In [ ]:
library_file_complete = library[library_cols]
library_file_complete.to_csv(library_to_add_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)

# view
display_df(library_file_complete)

### experiment annotations

In [ ]:
experiment = pd.read_csv(experiment_path_from_script, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN'], dtype=object)
display_df(experiment)

#### experiment and protocol details

In [ ]:
# this will give you the number of rows in the complete library file 
# this should be the number of annotated libraries
ann_lib = len(library_file_complete.index)
len(library_file_complete.index)

In [ ]:
# partial or total
experiment.loc[:,'experimentStatus'] = ''
experiment.loc[:,'projectTags'] = '' 
# see above cell, also can add as free text
experiment.loc[:,'numberOfAnnotatedLibraries'] = ann_lib

# these variables should already exist from above but if not can just add as free text
experiment.loc[:,'protocol'] = my_protocol
experiment.loc[:,'protocolType'] = my_protocolType

display_df(experiment)

#### paper and xrefs

In [ ]:
#experiment.loc[:,'GSE'] = ''
#experiment.loc[:,'Bioproject'] = '' 
experiment.loc[:,'PMID'] = ''
experiment.loc[:,'reference_url'] = ''
experiment.loc[:,'DOI'] = ''
#experiment.loc[:,'xrefs'] = ''

display_df(experiment)

#### comments

In [ ]:
#experiment.loc[:,'comment'] = ''

display_df(experiment)

#### save complete file

In [ ]:
experiment.to_csv(experiment_to_add_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)

### QA time

In [ ]:
library_to_add = pd.read_csv(library_to_add_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN'], dtype=object)
experiment_to_add = pd.read_csv(experiment_to_add_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN'], dtype=object)

#### to add things here

#### check columns match

In [ ]:
# pull from git and pull in library/experiment file
! git pull
git_library = pd.read_csv(git_library_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN'], dtype=object)
git_experiment = pd.read_csv(git_experiment_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN'], dtype=object)

# library file
if set(library_to_add.columns) == set(git_library.columns):
    print('The columns in the library file match')
else:
    print('The columns in the library file DO NOT MATCH')

# experiment file
if set(experiment_to_add.columns) == set(git_experiment.columns):
    print('The columns in the experiment file match')
else:
    print('The columns in the experiment file DO NOT MATCH')


# maybe to make this something more like "COLUMNS GOOD - LIBRARY" and "COLUMNS BAD - EXPERIMENT"

#### view files

In [ ]:
library_git_plus_new = pd.concat([git_library, library_to_add], ignore_index = True, sort = False)
library_git_plus_new.tail(n=10)

In [ ]:
experiment_git_plus_new = pd.concat([git_experiment, experiment_to_add], ignore_index = True, sort = False)
experiment_git_plus_new.tail(n=5)

### add annotations to git

In [ ]:
! git pull

In [ ]:
library_git_plus_new.to_csv(git_library_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)
experiment_git_plus_new.to_csv(git_experiment_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)
update_format(git_library_path)
update_format(git_experiment_path)

In [ ]:
! git status

In [ ]:
! git add $git_experiment_path $git_library_path

In [ ]:
! git commit -m $commit_message_exp

In [ ]:
! git push

### add annotation folder and script to git

In [ ]:
! git status

1. run first two cells (annotation summary)
2. export as html

In [ ]:
! git add $path_to_output

In [ ]:
! git commit -m $commit_message_py

In [ ]:
! git push